<a href="https://colab.research.google.com/github/fblondiaux/LGBIO2060-2020/blob/master/LGBIO2060_TP3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LGBIO2060 - Exercise session 3
Kalman filtering or how to implement the prior and likelihood dynamically? 


## Import and helper functions
**Please execute the cell below to initialize the notebook environment**

In [ ]:
#@title Imports
!pip install pykalman --quiet
import numpy as np
import matplotlib.pyplot as plt
import pykalman


In [ ]:
#@title Utility functions
def plot_my_system(state_evolution):
  """
  Do not edit this function...

  the aim of this function is to represent the time-evolution of a dynamical linear system
  Author : Antoine de Comite 
  """
  fig, ax = plt.subplots(figsize=(8,6))
  xlim = None
  ylim = None
  ax.scatter(state_evolution[0,:],state_evolution[1,:],c='m',s=100,alpha=0.7)
  ax.plot(state_evolution[0,:],state_evolution[1,:],LineWidth=2,c='k')
  ax.set_xlabel("x_1")
  ax.set_ylabel("x_2")
  ax.set(xlim=xlim)
  ax.set(ylim=ylim)
  plt.plot()
  return ax
def plot_my_system_with_obs(state_evolution,obs_evolution):
  """
  Do not edit this function...

  the aim of this function is to represent the time-evolution of a dynamical linear system
  Author : Antoine de Comite 
  """
  fig, ax = plt.subplots(figsize=(8,6))
  xlim = None
  ylim = None
  ax.scatter(state_evolution[0,:],state_evolution[1,:],c='m',s=100,alpha=0.7)
  ax.plot(state_evolution[0,:],state_evolution[1,:],LineWidth=2,c='k',label='Latent state')
  ax.plot(obs_evolution[0,:],obs_evolution[1,:],LineWidth=2,c='g',label='Observation')
  ax.set_xlabel("x_1")
  ax.set_ylabel("x_2")
  ax.set(xlim=xlim)
  ax.set(ylim=ylim)
  ax.legend()
  plt.plot()
  return ax
def plot_my_kalman_filter(state_evolution,obs_evolution,estimated_evolution):
  """
  DO NOT EDIT THIS FUNCTION
  author : antoine de Comite 
  """

  fig,ax = plt.subplots(figsize=(8,6))
  xlim = None; ylim = None
  ax.scatter(state_evolution[0,:-1],state_evolution[1,:-1],c='m',s=100,alpha=0.7)
  ax.plot(state_evolution[0,:-1],state_evolution[1,:-1],LineWidth=2,c='k',label='Latent state')
  ax.plot(obs_evolution[0,:-1],obs_evolution[1,:-1],LineWidth=2,c='g',label='Observation')
  ax.plot(estimated_evolution[0,:-1],obs_evolution[1,:-1],LineWidth=2,c='r',label='Estimation')
  ax.set_xlabel("x_1")
  ax.set_ylabel("x_2")
  ax.set(xlim=xlim)
  ax.set(ylim=ylim)
  ax.legend()
  plt.plot()
  return ax
def plot_gaze_data(data, img=None, ax=None):
    # overlay gaze on stimulus
    if ax is None:
        fig, ax = plt.subplots(figsize=(8, 6))

    xlim = None
    ylim = None
    if img is not None:
        ax.imshow(img, aspect='auto')
        ylim = (img.shape[0], 0)
        xlim = (0, img.shape[1])

    ax.scatter(data[:, 0], data[:, 1], c='m', s=100, alpha=0.7)
    ax.set(xlim=xlim, ylim=ylim)

    return ax
def plot_kf_state(kf, data, ax):
    mu_0 = np.ones(kf.n_dim_state)
    mu_0[:data.shape[1]] = data[0]
    kf.initial_state_mean = mu_0

    mu, sigma = kf.smooth(data)
    ax.plot(mu[:, 0], mu[:, 1], 'limegreen', linewidth=3, zorder=1)
    ax.scatter(mu[0, 0], mu[0, 1], c='orange', marker='>', s=200, zorder=2)
    ax.scatter(mu[-1, 0], mu[-1, 1], c='orange', marker='s', s=200, zorder=2)
import ipywidgets as widgets  # interactive display
%config InlineBackend.figure_format = 'retina'
# use NMA plot style
plt.style.use("https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/nma.mplstyle")
my_layout = widgets.Layout()

In [ ]:
#@title Data retrieval and loading
import io
import os
import hashlib
import requests

fname = "W2D3_mit_eyetracking_2009.npz"
url = "https://osf.io/jfk8w/download"
expected_md5 = "20c7bc4a6f61f49450997e381cf5e0dd"

if not os.path.isfile(fname):
  try:
    r = requests.get(url)
  except requests.ConnectionError:
    print("!!! Failed to download data !!!")
  else:
    if r.status_code != requests.codes.ok:
      print("!!! Failed to download data !!!")
    elif hashlib.md5(r.content).hexdigest() != expected_md5:
      print("!!! Data download appears corrupted !!!")
    else:
      with open(fname, "wb") as fid:
        fid.write(r.content)

def load_eyetracking_data(data_fname=fname):

  with np.load(data_fname, allow_pickle=True) as dobj:
    data = dict(**dobj)

  images = [plt.imread(io.BytesIO(stim), format='JPG')
            for stim in data['stimuli']]
  subjects = data['subjects']

  return subjects, images

## Tutorial objectives 

At the end of this tutorial, you should be able to explain the concepts of dynamical system and Kalman filter. You should also be able to use Kalman and EM algorithm on real data. 

# Section 1 - Dynamical systems

In this tutorial, we will be applying filtering methods to dynamical systems. A dynamical system is a system for which the time-evolution of any point can be described by a time-dependent function. Many systems that we encounter in our daily life are dynamical systems: eyes movements, reaching movements, walking, population dynamics,... 


A dynamical system can be observed through some noise and uncertainty similarly to a fixed variable (see the first two tutorials). Therefore, in order to obtain the best estimate of the latent state of the system, we can apply Bayes theorem at every time step. This is what we will do in this tutorial.


A dynamical system has the following continuous and discrete forms, respectively : 

\begin{eqnarray}
& & \\
\dot{x}\left(t\right) & = & A x\left(t\right) + \xi\left(t\right)\\
& & \\
x\left[t+1\right] & = & A x\left[t\right] + \xi\left[t\right] \\
& & \\
\end{eqnarray}

where $A$ is called the **state-transition matrix** matrix and $\xi\left(t\right)$ is Gaussian white motor noise generated from $\mathcal{N}\left(0,\Omega_{\text{motor}}\right)$. $\Omega_{\text{motor}}$ is the covariance matrix of the motor noise.

The latent state $x$ can have more than one entry, for example we may want to estimate the x- and y-positions of an annoying fly. In this case, the latent state will become a latent state vector $x = \left[x_1,x_2 \right]^T$ and the dynamical system can be rewritten : 


\begin{eqnarray}
&&\\
\begin{bmatrix}
  x_1[t+1] \\
  x_2[t+1]
  \end{bmatrix} &=& \begin{bmatrix}A_{11} & A_{12}\\ A_{21} & A_{22} \end{bmatrix}\begin{bmatrix}x_1[t]\\ x_2[t]\end{bmatrix} + \begin{bmatrix}\xi_1[t] \\ \xi_2[t] \end{bmatrix}
  &&\\
\end{eqnarray}

**Exercise 1 - Implement a function that defines the system mentionned above**


In this first exercise, you are asked to model the the dynamical system characterised by the equation above. 


Hints : 
* You can assume that the covariance matrix of the motor noise is diagonal and that all the entries of the diagonal are equal to $\sigma_{\text{motor}}$. (see the function np.random.multivariate_normal)
* You will be performing matrix multiplications, consider this while implementing your function...
* You can use the function *plot_my_system* to represent the time-evolution of your linear dynamical system





In [ ]:
def my_system(nsteps,x0,A,omega_motor,seed=2060):
  """
  my_system is a function that model the time-evolution of a linear dynamical system
  with state transition matrix A.
  Inputs : nsteps is the number of time steps to model
           x0 is the initial state (where we are starting from)
           A is the state-transition matrix
  Outputs : state_evolution is a numpy array that contains the time-evolution of 
            the state vector : (state size * nsteps)
  """
  # Set the random generator seed
  np.random.seed(seed)

  ########################
  #### Your code here ####
  ########################
  return state_evolution

# Run the lines below to test your code

nsteps = 50
x0 = np.array([1,1]).T
A = np.array([[1., 1.], [-(2*np.pi/20.)**2., .9]])
sigma_motor = 0.05
omega_motor = [[sigma_motor , 0],[0, sigma_motor]]

state_evolution = my_system(nsteps,x0,A,omega_motor)
plot_my_system(state_evolution)


Click [here](https://github.com/fblondiaux/LGBIO2060-2020/blob/master/Solutions/TP3Sol1.py) for solution.

*Example output:*

<img alt='Solution hint' align='left' width=413 height=300 src=https://raw.githubusercontent.com/fblondiaux/LGBIO2060-2020/master/Solutions/TP3-Ex1.png>


**What is the effect of the motor noise covariance on the behaviour of the system?**


Play with the widget below to explore this effect. 

In [ ]:
# @title
# @markdown Make sure you execute this cell to enable the widget!
my_layout.width = '450px'
@widgets.interact(
    sigma_motor=widgets.FloatSlider(0.05, min=0, max=1, step=0.05, layout=my_layout)

)

def sigma_motor(sigma_motor = 0.05):
    nsteps = 50
    x0 = np.array([1,1]).T
    A = np.array([[1., 1.], [-(2*np.pi/20.)**2., .9]])
    omega_motor = [[sigma_motor , 0],[0, sigma_motor]]
    state_evolution = my_system(nsteps,x0,A,omega_motor)
    plot_my_system(state_evolution)


---

However, as we saw in the previous tutorials, our sensory inputs are not perfect and it is impossible to know the real latent state; we can only get a rough estimate of it. Mathematically, we define the observation of the latent state of our dynamical system, $y[t]$ as follows: 


$\begin{eqnarray}
& & \\
y[t]& = & H x[t] + \eta[t]\\
 & & \\
\begin{bmatrix}y_1[t]\\ y_2[t]\end{bmatrix} & = &\begin{bmatrix}1 & 0 \\ 0 & 1\end{bmatrix} \begin{bmatrix}x_1[t] \\ x_2[t]\end{bmatrix} + \begin{bmatrix}\eta_1[t]\\ \eta_2[t]\end{bmatrix}
& &\\
\end{eqnarray}$

where $H$ is the observation matrix and $\eta[t]$ is a vector of Gaussian white sensory noise generated from $\mathcal{N}\left(0,\Omega_{\text{sensory}}\right)$. 

**Exercise 2 - Implement the observation of the latent state**

* you can use the function *plot_my_system_with_obs* for the plot

In [ ]:
def my_system_with_obs(nsteps,x0,A,H,omega_motor,omega_sensory):
  """
  my_system_with_obs is a function that model the time-evolution of the latent state 
  and its observation
  Inputs : nsteps is the number of time steps to model
           x0 is the initial state (where we are starting from)
           A is the state-transition matrix
           H is the observation matrix
           omega_motor is the variance of the motor noise
           sigma_sensory is the variance of the sensory noise
  Outputs : state_evolution is a numpy array that contains the time-evolution of 
            the state vector : (state size * nsteps)
            obs_evolution is a numpy array that contains the time-evolution of 
            the observation of the state vector : (state size * nsteps)
  """
  ######################
  ### your code here ###
  ######################
  
  return state_evolution,obs_evolution

# Run the lines below to test your code 
np.random.seed(2060)
nsteps = 50
x0 = np.array([1,1]).T
A = np.array([[1.,1.],[-(2*np.pi/20.)**2.,0.9]])
H = np.eye(2)
sigma_motor = 0.05
sigma_sensory = 0.02
omega_motor = [[sigma_motor , 0],[0, sigma_motor]]
omega_sensory = [[sigma_sensory , 0],[0, sigma_sensory]]
######################
### your code here ###
######################


*Example output:*
Click [here](https://github.com/fblondiaux/LGBIO2060-2020/blob/master/Solutions/TP3Sol2.py) for solution.

<img alt='Solution hint' align='left' width=413 height=300 src=https://raw.githubusercontent.com/fblondiaux/LGBIO2060-2020/master/Solutions/TP3-Ex2.png>

**Compared to the first and second tutorials, what does the green line correspond to (in terms of Bayes' theorem)?**




## Section 2 - Kalman filter or the optimal observation of continuous linear dynamical systems

Once again, our sensory inputs are not good enough to infer the position of the fly we are tracking. In order to have a better estimation of the fly's position at every time, we will use a continuous adaptation of Bayes' theorem: the **Kalman filter**. 

At every time step, the Kalman filter computes the weighted sum of the prior and the likelihood, weighted by a factor that characterises how trustable each term is. The result of this weighted sum will be the posterior distribution; this posterior distribution is the optimal estimator of the latent state of the system. 

The estimated state, denoted $\hat{x}$ is computed this way : 

$\begin{eqnarray}
& & \\
\hat{x}[t] & = & \text{trust}_{\text{prior}}x_{\text{prior}} + \text{trust}_{\text{likelihood}}x_{\text{likelihood}}\\
& & \\
\hat{x}[t+1] & = & A  \hat{x}[t]+ K[t]\left(y[t] - H \hat{x}[t]\right)\\
& & \\
\end{eqnarray}$

Where $K[t]$ is the Kalman gain evaluated at time $t$. These Kalman gains will be the weighting parameters that characterises how much we trust one source of information (prior and likelihood) over the other. Since these gains quantifies the trust we have in each source, they will be computed based on the covariances matrices of these two sources. The covariance matrix related to the prior is $\Omega_{\text{motor}}$ and the one related to the likelihood is $\Omega_{\text{sensory}}$. Therefore the gain of the kalman filter are recursively computed as follows:  

$\begin{eqnarray}
& & \\
K[t]& = &A\, \Sigma[t] \,H^T \left(H \,\Sigma[T]\,H^T+\Omega_{\text{motor}}\right)^{-1} \\
& & \\
\Sigma[t+1] & = & \Omega_{\text{sensory}} + \left(A-K[t]\,H\right)\Sigma[T]\,A^T
\end{eqnarray}$

Where $\Sigma[t] = \mathbb{C}\text{ov}\left\{\hat{x}[t]\right\}$ is the covariance matrix of the estimated state at time t. 


**Exercice 3 - Implement the function below to apply Kalman filter to the dynamical linear system**


In [ ]:
def my_kalman_filter(nsteps,x0,A,H,omega_motor,omega_sensory,seed=2060):
  """
  my_kalman_filter computes and applies the Kalman filter to the system seen above
  Inputs : nsteps is the number of time steps to model
           x0 is the initial state (where we are starting from)
           A is the state-transition matrix
           H is the observation matrix
  Outputs : latent_state is a numpy array that contains the time-evolution of 
             the state vector : (state size * nsteps)
            observed_state is a numpy array that contains the time-evolution of 
             the observation of the state vector : (state size * nsteps)
            estimated_state is a numpy array that contains the time-evolution of 
              the estimation of the state vector : (state size * nsteps)
            K kalman gains: (state size * state size * nsteps)
  """
  np.random.seed(seed)

  K = np.zeros((len(x0),len(x0),nsteps))
  Sigma = np.zeros((len(x0),len(x0),nsteps))

  ####################
  ## your code here ##
  ####################
  return latent_state,observed_state, estimated_state, K


# Run the lines below to test your code

# Parameters definition
nsteps = 50
A = np.array([[1.,1.],[-(2*np.pi/20.)**2.,0.9]])
H = np.eye(2)
x0 = np.array([1,1]).T
omega_motor = 0.05 * np.eye(len(x0))
omega_sensory = 0.02 * np.eye(len(x0))

####################
## your code here ##
####################



*Example output:*
Click [here](https://github.com/fblondiaux/LGBIO2060-2020/blob/master/Solutions/TP3Sol3.py) for solution.

<img alt='Solution hint' align='left' width=413 height=300 src=https://raw.githubusercontent.com/fblondiaux/LGBIO2060-2020/master/Solutions/TP3-Ex3.png>

**Before going to the next section, explore the effect that the initial parameters have on the system (A,H, x0, omega_motor and omega_sensory)**

##Section 3 - Application of Kalman to real data

Now that we have seen the mathematical ground of Bayesian integration and Kalman filtering, let's take a look at how they can be applied to real data. Tracking gaze is used for both fundamental and clinical research and is therefore a very active research field yet challenging because of the various sources of noise inherent to this kind of measurements. 

For this example, we will apply this method to a small subset of data from the [MIT Eyetracking Database](http://people.csail.mit.edu/tjudd/WherePeopleLook/index.html) [[Judd et al. 2009](http://people.csail.mit.edu/tjudd/WherePeopleLook/Docs/wherepeoplelook.pdf)]. This data was collected as part of an effort to model [visual saliency](http://www.scholarpedia.org/article/Visual_salience) -- given an image, can we predict where a person is most likely going to look.



In [ ]:
# load eyetracking data
subjects, images = load_eyetracking_data()

In this dataset, there are three different stimuli (images) and five different subjects. Each subject fixated in the center of the screen before the image appeared, then had a few seconds to freely look around. You can use the widget below to see how different subjects visually scanned the presented image. A subject ID of -1 will show the stimulus image without any overlayed gaze trace. 

Note that the images are rescaled below for display purposes, they were in their original aspect ratio during the task itself.

In [ ]:
#@markdown Make sure you execute this cell to enable the widget

@widgets.interact(subject_id=widgets.IntSlider(-1,min=-1,max=4),
                  image_id=widgets.IntSlider(0,min=0,max=2))
def plot_subject_trace(subject_id=-1, image_id=0):
  if subject_id==-1:
    subject = np.zeros((3,0,2))
  else:
    subject = subjects[subject_id]
  data = subject[image_id]
  img = images[image_id]

  fig, ax=plt.subplots()
  ax.imshow(img,aspect='auto')
  ax.scatter(data[:,0],data[:,1],c='m',s=100,alpha=0.7)
  ax.set(xlim=(0, img.shape[1]), ylim=(img.shape[0], 0))

**Fitting data with `pykalman`**


Now that we have data, we would like to use Kalman filtering to give us a better estimate of the true gaze. Up until this point we've known the parameters of our linear dynamical system, but here we need to estimate them from data directly. We will use the `pykalman` package to handle this estimation using the EM algorithm.

Before exploring fitting models with `pykalman` it's worth pointing out some naming conventions used by the library:

$$
\begin{align}
A &: \texttt{transition_matrices} & 
\Omega_{\text{motor}} &: \texttt{transition_covariance}\\
H &:\texttt{observation_matrices} &
\Omega_{\text{sensory}} &:\texttt{observation_covariance}\\
\mu_0 &: \texttt{initial_state_mean} & \Sigma_0 &: \texttt{initial_state_covariance}
\end{align}
$$

The first thing we need to do is provide a guess at the dimensionality of the latent state. Let us start by assuming the dynamics line-up directly with the observation data (pixel x,y-coordinates), and so we have a state dimension of 2.

We also need to decide which parameters we want the EM algorithm to fit. In this case, we will let the EM algorithm discover the dynamics parameters i.e. the $A$, $\Omega_{\text{motor}}$, $H$, and $\Omega_{\text{sensory}}$ matrices.

**Exercise** : Set up our `pykalman` `KalmanFilter` object with these settings using the code below. You can find the help for this library [here](https://pykalman.github.io/#kalman-filter-user-s-guide).

In [ ]:
# set up our KalmanFilter object and tell it which parameters we want to
# estimate
np.random.seed(1)

n_dim_obs = 2
n_dim_state = 2
######################
### your code here ###
######################
kf = ...

Click [here](https://github.com/fblondiaux/LGBIO2060-2020/blob/master/Solutions/TP3Sol4.py) for solution.

Because we know from the reported experimental design that subjects fixated in the center of the screen right before the image appears, we can set the initial starting state estimate $\mu_0$ as being the center pixel of the stimulus image (the first data point in this sample dataset) with a correspondingly low initial noise covariance $\Sigma_0$. Once we have everything set, it's time to fit some data.

In [ ]:
# Choose a subject and stimulus image
subject_id = 1
image_id = 2
data = subjects[subject_id][image_id]

# Provide the initial states
kf.initial_state_mean = #fill here
kf.initial_state_covariance = #fill here

# Estimate the parameters from data using the EM algorithm
#fill here

print(f'F =\n{kf.transition_matrices}')
print(f'Q =\n{kf.transition_covariance}')
print(f'H =\n{kf.observation_matrices}')
print(f'R =\n{kf.observation_covariance}')

We see that the EM algorithm has found fits for the various dynamics parameters. One thing you will note is that both the state and observation matrices are close to the identity matrix, which means the x- and y-coordinate dynamics are independent of each other and primarily impacted by the noise covariances.

We can now use this model to smooth the observed data from the subject. In addition to the source image, we can also see how this model will work with the gaze recorded by the same subject on the other images as well, or even with different subjects.

Below are the three stimulus images overlayed with recorded gaze in magenta and smoothed state from the filter in green, with gaze begin (orange triangle) and gaze end (orange square) markers. 

In [ ]:
#@title

#@markdown Make sure you execute this cell to enable the widget!

@widgets.interact(subject_id=widgets.IntSlider(1, min=0, max=4))
def plot_smoothed_traces(subject_id=0):
  subject = subjects[subject_id]
  fig, axes = plt.subplots(ncols=3, figsize=(18, 4))
  for data, img, ax in zip(subject, images, axes):
    ax = plot_gaze_data(data, img=img, ax=ax)
    plot_kf_state(kf, data, ax)

Why do you think one trace from one subject was sufficient to provide a decent fit across all subjects? If you were to go back and change the subject_id and/or image_id for when we fit the data using EM, do you think the fits would be different?

Finally, recall that the orignial task was to use this data to help devlop models of visual salience. While our Kalman filter is able to provide smooth estimates of observed gaze data, it's not telling us anything about *why* the gaze is going in a certain direction. In fact, if we sample data from our parameters and plot them, we get what amounts to a random walk.

In [ ]:
kf_state, kf_data = kf.sample(len(data))
ax = plot_gaze_data(kf_data, img=images[2])
plot_kf_state(kf, kf_data, ax)

This should not be surprising, as we have given the model no other observed data beyond the pixels at which gaze was detected. We expect there is some other aspect driving the latent state of where to look next other than just the previous fixation location.

In summary, while the Kalman filter is a good option for smoothing the gaze trajectory itself, especially if using a lower-quality eye tracker or in noisy environmental conditions, a linear dynamical system may not be the right way to approach the much more challenging task of modeling visual saliency.
